### Импорты

In [1]:
import pandas as pd
from scipy import stats as st
import math as mth

In [2]:
ab = pd.read_csv('analytical_skills.csv')
display(ab)
display(ab.info())

,Ветка,Вертикаль,Страница товара (пользователей),Число покупателей
0,Control,Смартфоны,12137,1972
1,Variation,Смартфоны,12289,2065
2,Control,Обувь,5070,268
3,Variation,Обувь,5273,311
4,Control,Компьютеры,8239,772
5,Variation,Компьютеры,8092,752
6,Control,Бытовая техника,7164,709
7,Variation,Бытовая техника,7247,702
8,Control,Одежда,5106,272
9,Variation,Одежда,5115,265


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Ветка                            12 non-null     object
 1   Вертикаль                        12 non-null     object
 2   Страница товара (пользователей)  12 non-null     int64 
 3   Число покупателей                12 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 512.0+ bytes


None

Стоит переименовать названия столбцов в таблице, чтобы при дальнейшем анализе к ним было удобно обращаться:

In [3]:
ab.columns = ['branch', 'category', 'item_page_views', 'purchases']

#### Вывод:
Был проведен правильный A/B тест, где пользователи, просматривающие товары в 6 различных категориях, были поделены на 2 группы, контрольную (Control) и тестовую (Variation). В каждой категории для каждой из групп, нам известно, сколько пользователей просматривали эту категорию (item_page_views) и сколько зашедших пользователей совершили покупку в этой категории (purchases).
* Столбцам присвоены правильные типы данных
* Для удобства столбцы были переименованы

### Гипотеза

Если человек, видит, что товар популярен, то получает больше доверия к товару/продавцу (social proof). Поэтому если показать информацию о том, сколько раз этот товар был куплен, то это может увеличить конверсию.

**Задача** - проанализировать результаты теста и выяснить, стоит ли внедрять эту фитчу.

### Предобработка

Отфильтруем данные по группам:

In [4]:
sample_A = ab[ab['branch'] == 'Control'].reset_index()
sample_B = ab[ab['branch'] == 'Variation'].reset_index()

Создадим функцию, которая будет проверять гипотезы о равенстве долей. На вход она будет принимать выборки двух контрольных групп, категорию, по которомй будут сравниваться группы и критический уровень статистической значимости:

In [5]:
def z_test(sample_1, sample_2, category_name, alpha=0.05):
    
    # критический уровень статистической значимости
    alpha = alpha
    
    # число успехов это количество покупок товара в категории
    successes_1 = sample_1[sample_1['category'] == category_name]['purchases'].item()
    successes_2 = sample_2[sample_2['category'] == category_name]['purchases'].item()
    
    # число попыток это кол-во просмотров страницы товара в категории
    trials_1 = sample_1[sample_1['category'] == category_name]['item_page_views'].item()
    trials_2 = sample_1[sample_1['category'] == category_name]['item_page_views'].item()
    
    #пропорция успехов в первой группе:
    p1 = successes_1 / trials_1
    #пропорция успехов во второй группе:
    p2 = successes_2 / trials_2
    #пропорция успехов в комбинированном датасете:
    p_combined  = (successes_1 + successes_2) / (trials_1 + trials_2)
    #разница пропорций в датасетах
    difference = p1 - p2
    
    #считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(p_combined  *  (1 - p_combined ) * (1 / trials_1 + 1 / trials_2))
    
    #задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1) 
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    print('Группа', sample_1['branch'][0],'насчитывает в категории', category_name,
          f'{successes_1} купивших товар пользователей, 'f'доля купивших: {p1:.1%}')
    print('Группа', sample_2['branch'][0],'насчитывает в категории', category_name,
          f'{successes_2} купивших товар пользователей, 'f'доля купивших: {p2:.1%}')
    print()
    
    print('p-значение: ', p_value)
    if (p_value < alpha):
        print("Отвергаем нулевую гипотезу, между долями есть значимая разница")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")

### A/B тест
* **H0** = Доли купивших товар пользователей двух сравниваемых контрольных групп `равны`
* **H1** = Доли купивших товар пользователей двух сравниваемых контрольных групп `различны`

In [6]:
z_test(sample_A, sample_B, 'Смартфоны', alpha=0.05)

Группа Control насчитывает в категории Смартфоны 1972 купивших товар пользователей, доля купивших: 16.2%
Группа Variation насчитывает в категории Смартфоны 2065 купивших товар пользователей, доля купивших: 17.0%

p-значение:  0.10892026381492914
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [7]:
z_test(sample_A, sample_B, 'Обувь', alpha=0.05)

Группа Control насчитывает в категории Обувь 268 купивших товар пользователей, доля купивших: 5.3%
Группа Variation насчитывает в категории Обувь 311 купивших товар пользователей, доля купивших: 6.1%

p-значение:  0.06571931463689529
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [8]:
z_test(sample_A, sample_B, 'Компьютеры', alpha=0.05)

Группа Control насчитывает в категории Компьютеры 772 купивших товар пользователей, доля купивших: 9.4%
Группа Variation насчитывает в категории Компьютеры 752 купивших товар пользователей, доля купивших: 9.1%

p-значение:  0.590723446569001
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [9]:
z_test(sample_A, sample_B, 'Бытовая техника', alpha=0.05)

Группа Control насчитывает в категории Бытовая техника 709 купивших товар пользователей, доля купивших: 9.9%
Группа Variation насчитывает в категории Бытовая техника 702 купивших товар пользователей, доля купивших: 9.8%

p-значение:  0.8444014702131721
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [10]:
z_test(sample_A, sample_B, 'Одежда', alpha=0.05)

Группа Control насчитывает в категории Одежда 272 купивших товар пользователей, доля купивших: 5.3%
Группа Variation насчитывает в категории Одежда 265 купивших товар пользователей, доля купивших: 5.2%

p-значение:  0.7563007547179244
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


In [11]:
z_test(sample_A, sample_B, 'Парфюмерия', alpha=0.05)

Группа Control насчитывает в категории Парфюмерия 135 купивших товар пользователей, доля купивших: 5.5%
Группа Variation насчитывает в категории Парфюмерия 176 купивших товар пользователей, доля купивших: 7.1%

p-значение:  0.016312067705062105
Отвергаем нулевую гипотезу, между долями есть значимая разница


### Вывод:
Из всех 6-ти проведенных стат. тестов, только в категории "Парфюмерия" была найдена значимая разница в доле купивших между двумя сравниваемыми группами. Это может быть связанно с тем, что парфюмерную продукцию трудно оценить онлайн. Когда мы выбираем одежду или технику, основными факторами влиящими на наш выбор становятся внешний вид и характеристики, тогда как приятный цвет и форма флакона хоть и сделают парфюм привлекательнее, но абсолютне ничего не скажут о его характерной черте - аромате. При выборе парфюма мы во многом опираемся на личный опыт или рекомендации, что и могло сработать в этом случае. Покупатель может выбирать какой-то новый аромат для себя или в подарок, опираясь на популярность и отзывы в интернете, получая больше доверия к товару. Исходя из этого может появиться дополнительная гипотеза: если человек видит, что товар, который нельзя оценить визуально или объективно сравнить с другими такими товарами (парфюм, еда для домашних питомцев, пищевая продукция), популярен среди других пользователей, то получает больше доверия к товару или продавцу. Это можно проверить схожим A/B тестом с отличием лишь в сравниваемых категориях. 

В остальных случаях, добавление блока, с информацией о том, сколько раз этот товар был куплен, никак не повлияло на поведение и покупки пользователей. Внедрять эту фичу, на данном этапе, не стоит. 




_* если делать поправку на множественное сравнение (Бонферонни или Шидак) - статистическая значимость во всех тестах не позволяет отвергнуть нулевую гипотезу_